In [4]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import rasterio as rio

In [90]:
class SceneTiler():
    def __init__(self, scenes_dir, tile_shape):
        self.root = pathlib.Path(scenes_dir)
        self.tile_shape = tile_shape
        self.tile_path = self.root.parent / f"{self.root.stem}-crop{self.tile_shape[0]}-{self.tile_shape[1]}"

    def _get_2d_window_coords(self, scene_shape, tile_shape) -> list:
        window_coords = list()
        (num_windows_x, remainder_x), (num_windows_y, remainder_y) = divmod(scene_shape[0], tile_shape[0]), divmod(scene_shape[1], tile_shape[1])

        if not remainder_x:
            num_windows_x -= 1
        
        if not remainder_y:
            num_windows_y -= 1

        for y_incr in range(num_windows_y+1):
            for x_incr in range(num_windows_x+1):

                x_coordinate = x_incr * tile_shape[0]
                y_coordinate = y_incr * tile_shape[1]

                window_coords.append((x_coordinate, y_coordinate))
                if x_incr == num_windows_x:
                    window_coords.append((scene_shape[0] - tile_shape[0], y_coordinate))
                if y_incr == num_windows_y:
                    window_coords.append((x_coordinate, scene_shape[1] - tile_shape[0]))
        return window_coords

    def _scene_tiler(self, scene_path) -> None:
        with rio.open(scene_path) as scene:
            window_clips = self._get_2d_window_coords(scene.shape, self.tile_shape)
            scene_name = scene.name.split('/')[-1].split('.')[0]
            profile = scene.profile

            for row, col in window_clips:
                window = rio.windows.Window(row, col, self.tile_shape[0], self.tile_shape[1])        
                transform = scene.window_transform(window)
                profile.update({
                    'width': self.tile_shape[0],
                    'height': self.tile_shape[1],
                    'transform': transform
                })
                tile_path = self.tile_path / f"{scene_name}_{row}_{col}.tif"
                with rio.open(tile_path.as_posix(), 'w', **profile) as tile:
                    tile.write(scene.read(window = window))

    def tile(self) -> None:
        self.tile_path.mkdir(parents = True, exist_ok = True)
        for scene_path in self.root.iterdir():
            self._scene_tiler(scene_path.as_posix())
            break

In [91]:
DATA = pathlib.Path("/media/sambhav/30AC4696AC46568E/datasets/inaria")
TRAIN  = DATA / "train"

In [92]:
image_tiler = SceneTiler(TRAIN / "images", (256, 256))
target_tiler = SceneTiler(TRAIN / "gt", (256, 256))

image_tiler.tile()
target_tiler.tile()

In [94]:
(2 * 5 * 36 * 3) / 60

18.0